In [1]:
# %load_ext lab_black

In [2]:
from pathlib import Path

# Essa variável vai apontar para onde os dados moram
# no caso - uma pasta pra cima da pasta atual, e
# então na pasta "sample_data"
# ref: https://docs.python.org/pt-br/3/library/pathlib.html

pasta_dados = Path().absolute().parent / "sample_data"

Alerta: Esse notebook contém código escrito em Português. Se você se surpreende com isso ou quer saber porquê esse aviso está aqui, [leia](https://notasdobidu.com/eca-o-bidu-ta-programando-em-portugues/)

# Lab de ETL - Nível 01

Nessa primeira etapa vamos carregar arquivos CSV e fazer alguma limpezas, em memória mesmo

## 01 - Listando e carregando os dados de exemplo

Quantos arquivos temos na pasta?

In [3]:
for filename in pasta_dados.iterdir():
    print(filename.name)

2021-02-01.csv
2021-02-09.csv
2021-02-12.csv
2021-02-13.csv
2021-02-15.csv
2021-02-19.csv
2021-02-22.csv
2021-02-28.csv


Carregando um desses arquivos:

In [4]:
import pandas as pd

In [5]:
dados = pd.read_csv(pasta_dados / "2021-02-01.csv")

In [6]:
dados.head()

,Núm.,Descrição,Qtd. Comercial,Unid. Comercial,Valor Líquido do Item,Informações Adicionais do Produto,Cód. Produto,Cód. GTIN,Cód. NCM,Código Especificador da Substituição Tributária,...,Valor de Deduções para ISSQN,Valor da Base de Cálculo do ISSQN,Alíquota do ISSQN,Valor do ISSQN,Item da Lista de Serviços,Código do Município do Fato Gerador do ISSQN,Código de Tributação pelo ISSQN do Município,Natureza da Operação de ISSQN,Incentivo Fiscal do ISSQN,Valor Aproximado dos Tributos do Produto ou Serviço – Lei 12741/12
0,1,"CHA LIPTON 1,5L PESSEGO","1,0000",UN,"7,15",Não Informado,7891042000195,7891042000195,22021000,1711100,...,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,2 - Não,"2,34"
1,2,"SUCO AURORA 1,5L UVA TTO","1,0000",UN,"17,29",Não Informado,7891141019838,7891141019838,20096100,1701000,...,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,2 - Não,"3,83"
2,3,CREME LEITE ITAMBE 300g,"1,0000",UN,"4,99",Não Informado,7896051114086,7896051114086,4015029,1701902,...,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,2 - Não,"0,54"
3,4,LEITE SHEFA 1L GF.INTEGR,"1,0000",UN,"3,99",Não Informado,7896185312396,7896185312396,4012010,1701601,...,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,2 - Não,"0,16"
4,5,PALMITO LUPE 300g P.TOLE,"1,0000",UN,"11,89",Não Informado,7898944191778,7898944191778,20089100,1709500,...,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,2 - Não,"2,63"


Como podemos ver, temos muitas colunas e os dados precisam de uma limpeza. Vamos fazer o fluxo de "ETL" completo com um arquivo só e então expandir para todos os arquivos na pasta.

Nós já temos a primeira etapa - carregar os dados é o início, é o "E" de ETL. Portanto para o nosso caso, essa etapa consiste simplesmente de chamar o pandas para carregar o CSV

## Tratamentos

Agora vamos para segunda etapa, o T - "tract". Essa etapa consiste em tratamentos que queremos fazer nos dados.

Primeiro, vamos deixar só as colunas que queremos

In [7]:
colunas = [
    "Descrição",
    "Qtd. Comercial",
    "Unid. Comercial",
    "Valor Líquido do Item",
    "Cód. Produto",
]

In [8]:
dados = dados[colunas]

In [9]:
dados.head()

,Descrição,Qtd. Comercial,Unid. Comercial,Valor Líquido do Item,Cód. Produto
0,"CHA LIPTON 1,5L PESSEGO","1,0000",UN,"7,15",7891042000195
1,"SUCO AURORA 1,5L UVA TTO","1,0000",UN,"17,29",7891141019838
2,CREME LEITE ITAMBE 300g,"1,0000",UN,"4,99",7896051114086
3,LEITE SHEFA 1L GF.INTEGR,"1,0000",UN,"3,99",7896185312396
4,PALMITO LUPE 300g P.TOLE,"1,0000",UN,"11,89",7898944191778


Nosso primeiro tratamento está feito. Agora podemos fazer uma etapa adicional, conferir os tipos:

In [10]:
dados.dtypes

Descrição                object
Qtd. Comercial           object
Unid. Comercial          object
Valor Líquido do Item    object
Cód. Produto              int64
dtype: object

E converter as colunas para os tipos corretos:

In [11]:
dados["Qtd. Comercial"] = pd.to_numeric(dados["Qtd. Comercial"])
dados["Valor Líquido do Item"] = pd.to_numeric(dados["Valor Líquido do Item"])

ValueError: Unable to parse string "1,0000" at position 0

Tivemos problemas com as duas colunas! Isso é por causa da vírgula, que não é reconhecida como separador decimal. Vamos primeiro converter todas para pontos:

In [12]:
dados["Qtd. Comercial"] = dados["Qtd. Comercial"].str.replace(",", ".")
dados["Valor Líquido do Item"] = dados["Valor Líquido do Item"].str.replace(",", ".")

In [13]:
dados.head()

,Descrição,Qtd. Comercial,Unid. Comercial,Valor Líquido do Item,Cód. Produto
0,"CHA LIPTON 1,5L PESSEGO",1.0000,UN,7.15,7891042000195
1,"SUCO AURORA 1,5L UVA TTO",1.0000,UN,17.29,7891141019838
2,CREME LEITE ITAMBE 300g,1.0000,UN,4.99,7896051114086
3,LEITE SHEFA 1L GF.INTEGR,1.0000,UN,3.99,7896185312396
4,PALMITO LUPE 300g P.TOLE,1.0000,UN,11.89,7898944191778


In [14]:
dados["Qtd. Comercial"] = pd.to_numeric(dados["Qtd. Comercial"])
dados["Valor Líquido do Item"] = pd.to_numeric(dados["Valor Líquido do Item"])

In [15]:
dados

,Descrição,Qtd. Comercial,Unid. Comercial,Valor Líquido do Item,Cód. Produto
0,"CHA LIPTON 1,5L PESSEGO",1.000,UN,7.15,7891042000195
1,"SUCO AURORA 1,5L UVA TTO",1.000,UN,17.29,7891141019838
2,CREME LEITE ITAMBE 300g,1.000,UN,4.99,7896051114086
3,LEITE SHEFA 1L GF.INTEGR,1.000,UN,3.99,7896185312396
4,PALMITO LUPE 300g P.TOLE,1.000,UN,11.89,7898944191778
...,...,...,...,...,...
57,GOIABA VERMELHA - O QUIL,0.445,kg,2.22,172
58,MAMAO PAPAYA - O QUILO,0.545,kg,2.72,193
59,ALFACE MIMOSA RICELFOLHA,1.000,UN,2.99,7898407900046
60,RUCULA,1.000,UN,3.49,11289


Agora, vamos deixar os nomes dos produtos mais amigáveis, convertendo todos em "títulos":

In [16]:
dados["Descrição"].str.title()

0      Cha Lipton 1,5L Pessego
1     Suco Aurora 1,5L Uva Tto
2      Creme Leite Itambe 300G
3     Leite Shefa 1L Gf.Integr
4     Palmito Lupe 300G P.Tole
                ...           
57    Goiaba Vermelha - O Quil
58      Mamao Papaya - O Quilo
59    Alface Mimosa Ricelfolha
60                      Rucula
61              Cebolinha Maco
Name: Descrição, Length: 62, dtype: object

In [17]:
dados["Descrição"] = dados["Descrição"].str.title()

In [21]:
dados[dados["Descrição"].str.contains("- O Q")]

,Descrição,Qtd. Comercial,Unid. Comercial,Valor Líquido do Item,Cód. Produto
47,Banana Prata - O Quilo,0.770,kg,5.00,167
51,Cenoura - O Quilo,0.325,kg,0.97,239
52,Pimentao Verde - O Quilo,0.180,kg,0.72,230
53,Maracuja Azedo - O Quilo,0.365,kg,3.28,199
54,Cebola Nacional - O Quil,0.960,kg,4.79,129
55,Tomate Debora - O Quilo,0.425,kg,1.70,288
56,Cebola Roxa - O Quilo,0.265,kg,2.65,1907
57,Goiaba Vermelha - O Quil,0.445,kg,2.22,172
58,Mamao Papaya - O Quilo,0.545,kg,2.72,193


In [31]:
def remove_quilo(texto):
    if (" - O Q") in texto:
        return texto[:texto.index(" - O Q")]
    return texto

In [32]:
remove_quilo("Banana Prata - O Quilo")

'Banana Prata'

In [33]:
remove_quilo("Cebola Nacional - O Quil")

'Cebola Nacional'

In [34]:
dados["Descrição"].map(remove_quilo)

0      Cha Lipton 1,5L Pessego
1     Suco Aurora 1,5L Uva Tto
2      Creme Leite Itambe 300G
3     Leite Shefa 1L Gf.Integr
4     Palmito Lupe 300G P.Tole
                ...           
57             Goiaba Vermelha
58                Mamao Papaya
59    Alface Mimosa Ricelfolha
60                      Rucula
61              Cebolinha Maco
Name: Descrição, Length: 62, dtype: object

In [35]:
dados["Descrição"] = dados["Descrição"].map(remove_quilo)

In [36]:
dados.head()

,Descrição,Qtd. Comercial,Unid. Comercial,Valor Líquido do Item,Cód. Produto
0,"Cha Lipton 1,5L Pessego",1.0,UN,7.15,7891042000195
1,"Suco Aurora 1,5L Uva Tto",1.0,UN,17.29,7891141019838
2,Creme Leite Itambe 300G,1.0,UN,4.99,7896051114086
3,Leite Shefa 1L Gf.Integr,1.0,UN,3.99,7896185312396
4,Palmito Lupe 300G P.Tole,1.0,UN,11.89,7898944191778


## Load - Carregando os dados!

A etapa final é de carregamento de dados. Normalmente, essa etapa seria voltada
para carregar os dados de um banco de dados, mas nesse caso, vamos simplesmente
carregar os dados de volta em um arquivo csv!

Utilizando Pandas isso é bem direto:

In [37]:
dados.head()

,Descrição,Qtd. Comercial,Unid. Comercial,Valor Líquido do Item,Cód. Produto
0,"Cha Lipton 1,5L Pessego",1.0,UN,7.15,7891042000195
1,"Suco Aurora 1,5L Uva Tto",1.0,UN,17.29,7891141019838
2,Creme Leite Itambe 300G,1.0,UN,4.99,7896051114086
3,Leite Shefa 1L Gf.Integr,1.0,UN,3.99,7896185312396
4,Palmito Lupe 300G P.Tole,1.0,UN,11.89,7898944191778


In [38]:
dados.to_html("dados.html")

In [39]:
dados.to_csv("dados.csv")

## Juntando tudo

In [46]:
def extract(arquivo):
    return pd.read_csv(arquivo)

In [56]:
def tract(dados):
    colunas = [
        "Descrição",
        "Qtd. Comercial",
        "Unid. Comercial",
        "Valor Líquido do Item",
        "Cód. Produto",
    ]
    dados = dados[colunas].copy()
    dados["Qtd. Comercial"] = dados["Qtd. Comercial"].str.replace(",", ".")
    dados["Valor Líquido do Item"] = dados["Valor Líquido do Item"].str.replace(",", ".")
    dados["Qtd. Comercial"] = pd.to_numeric(dados["Qtd. Comercial"])
    dados["Valor Líquido do Item"] = pd.to_numeric(dados["Valor Líquido do Item"])
    dados["Descrição"] = dados["Descrição"].str.title()
    dados["Descrição"] = dados["Descrição"].map(remove_quilo)
    return dados

In [57]:
def load(dados, nome_arquivo):
    dados.to_csv(f"saída_{nome_arquivo}.csv")

In [58]:
for filename in pasta_dados.iterdir():
    nome = filename.name[:-4]
    print(f"Processando {nome}")
    dados = extract(filename)
    dados = tract(dados)
    load(dados, nome)

Processando 2021-02-01
Processando 2021-02-09
Processando 2021-02-12
Processando 2021-02-13
Processando 2021-02-15
Processando 2021-02-19
Processando 2021-02-22
Processando 2021-02-28
